In [ ]:
import requests
import json
import pandas as pd
import numpy as np
import re
import os
import duckdb
from shapely.geometry import Point
import geopandas as gpd
from modules import geo_calculations
pd.options.display.max_columns = None
pd.options.display.max_rows = None

Data places

In [ ]:


response = requests.get('https://datos.madrid.es/egob/catalogo/200761-0-parques-jardines.json')
results_api = response.json()
list_api_garden = results_api['@graph']
df_garden = pd.DataFrame(list_api_garden)
df_garden_location = df_garden["location"].apply(pd.Series)
df_garden_address = df_garden["address"].apply(pd.Series)
df_garden_type = df_garden["@type"].str.replace('https://datos.madrid.es/egob/kos/entidadesYorganismos/', '')
both_df_garden = pd.concat([df_garden[['title']], df_garden_location, df_garden_address, df_garden_type], axis = 1)
cleaning_df_garden = both_df_garden[['title', '@type', 'locality', 'postal-code', 'street-address', 'latitude', 'longitude']]





Data stations

In [ ]:
db_bicimad = duckdb.connect('bicimad1.db')
df_bicimad = db_bicimad.sql('select * from bicimad_stations').df()
df_bicimad[['longitude', 'latitude']] = df_bicimad["geometry.coordinates"].apply(lambda x: pd.Series(str(x).replace('[','').replace(']','').split(",")))
df_bicimad = df_bicimad[['name', 'address', 'latitude', 'longitude',]]




distancias

In [ ]:
%%time
distance=[]
for index, row in cleaning_df_garden.iterrows():
    for index, j in df_bicimad.iterrows():
        distance.append(distance_meters(float(row['latitude']), float(row['longitude']), float(j['latitude']), float(j['longitude'])))
        
list_df_coordinates=[]
count=0
list_list = []
for i in distance:
    if count != 264:
        count = count + 1
        list_list.append(i)
    if count == 264: # = rows of stations --> df_bicimad(rows)
        count = 0
        list_df_coordinates.append(list_list)
        list_list = []
                
df_coordinates = pd.DataFrame(list_df_coordinates)
        
def wrangcoor(coor):
    coor.astype(str)
    coor.applymap(re.sub('[^0-9\.]+', '', coor))
    coor.astype(float)
    coorm = coor.min(axis=1)
    coorm.to_dict()
    coor.to_dict()
    dict_min_station = {}
    for keys1, values1 in coor.items():
        for key_a, value_a in values1.items():
            for key_min, value_min in coorm.items():
                if value_a == value_min:
                    coorm[key_min] = (value_min, keys1)
                    coorm = dict(sorted(coorm.items()))        
                    df_min_station=pd.DataFrame.from_dict(coorm, orient='index')        
                    df_min_station.rename(columns={1: "index_garden"}, inplace=True)     
                    return df_min_station
        

df_bicimad['index'] = df_bicimad.index


df_bicimad.rename(columns={"index": "1a"}, inplace=True)


list_min_station_names = {}
count = 0
for index, row1 in df_min_station.iterrows():
    for index, row2 in df_bicimad.iterrows():
        if row1['1a'] == row2['1a']:
            list_min_station_names[count]= (row2['name'], row2['address'])
            count = count +1

df_min_station_names = pd.DataFrame(list_min_station_names)            
            
df_min_station_names = df_min_station_names.transpose()            
            
df_min_station_names.rename(columns={0: "name1", 1:'address1'}, inplace=True)            

df_min_station_names1 = df_min_station_names['name1'].apply(lambda x: pd.Series(str(x).split('-')))

df_min_station_names1.rename(columns={0: "numberof", 1:'name2', 2:'null'}, inplace=True)

df_final = pd.concat([cleaning_df_garden, df_min_station_names, df_min_station_names1], axis=1)

df_final = df_final[['title', '@type', 'street-address', 'name2', 'address1']]

df_final = df_final.rename(columns={'title': 'Place of interest', '@type': 'Type of place', 'street-address': 'Place address', 'name2': 'BiciMAD station', 'address1': 'Station location'})

df_final["Type of place"].fillna("ParquesJardines", inplace = True)







Reporting

In [ ]:
def result(dff):
    dff.to_csv('./data/gardens.csv')
